# Text Representation

Transform a pre-processed text into suitable numerical form and fed into ML algorithm for further process is called feature extraction or text representation.
Feature extraction is common step in any ml problem such as image, video, audio. 

 - Images will be transform into matrix representation based on their pixel values.
 - Video also similar, video is just a collection of frames where each frame is an image.  so the video represent as a sequential collection of matrices. 
 - Audio usually transmit as waves. so represent this mathematically, sampled wave amplitude will be 
recorded. this will give array representation of the sound waves.

Text representation approach classified into 4 categories

 - Basic vectorization approaches
 - Distributed representations
 - Universal language representation
 - Handcrafted Features

##### Text which represent by vectors of numbers is called vector space model. It's simple model used for representing any text blob. It's fundamental to many NLP operations like info-retrieval, scoring the documents etc.,  

## Basic vectorization approaches
Match each word in the vocabulary of the text corpus to a unique ID(integer). Then represent sentence in the corpus as a v-dimensional vector. 

### One- Hot Encoding
In this method, each word w in the corpus given a unique integer ID, It's between 1 & |V|. V is the set of the corpus vocabulary. Each word is then represent by a V-dimensional binary vector. 

- One hot encoding is intuitive to undetrstand and straight forward to implement
- Size of the one-hot vector is directly proportional to size of the vocabulary. so for large coprora it is computationaly ineffiecient to compute and store.
- This doesn't give fixed-length representation.
- It treats words as atomic unit and poor at capturing the meaning of the word in relation to other words.(run, ran, apple)
- Out of vocabulary problem. 


In [1]:
# List of sentences
sent_list = ["i read newspaper yesterday.", "I watched TV Today.", "john read newspaper and watched TV today."]

pre_process_list = [i.lower().replace(".","") for i in sent_list]

In [2]:
print(pre_process_list)

['i read newspaper yesterday', 'i watched tv today', 'john read newspaper and watched tv today']


In [4]:
# Build vocabulary set for the pre-processed list
vocab = {}
count = 0
for i in pre_process_list:
    for w in i.split():
        if w not in vocab:
            count = count + 1
            vocab[w] = count

In [5]:
print(vocab)

{'i': 1, 'read': 2, 'newspaper': 3, 'yesterday': 4, 'watched': 5, 'tv': 6, 'today': 7, 'john': 8, 'and': 9}


In [21]:
def get_onehot_encoding(text):
    """
        Generate one hot encoding for string based on vocab set. 
        If word exisst, it's representation in vocab will be returned.
        if not, a list of zero returned.
    """
    one_hot_encoded = []
    for w in text.split():
        temp = [0]*len(vocab)
        if w in vocab:
            temp[vocab[w]-1] = 1# -1 because array indexing start from 0
        one_hot_encoded.append(temp)
    return one_hot_encoded

In [22]:
get_onehot_encoding(pre_process_list[0])

[[1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0]]

In [23]:
# Using Skikit learn
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [24]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [32]:
nest_list = [i.split() for i in pre_process_list]

word_list = [ item for elem in nest_list for item in elem]

print(word_list)

['i', 'read', 'newspaper', 'yesterday', 'i', 'watched', 'tv', 'today', 'john', 'read', 'newspaper', 'and', 'watched', 'tv', 'today']


In [34]:
# Label Encodeing
label_encoder = LabelEncoder()
integer_encoded_values = label_encoder.fit_transform(word_list)

print(integer_encoded_values)

[1 4 3 8 1 7 6 5 2 4 3 0 7 6 5]


In [ ]:
# One Hot Encoding
onehot_encoder = OneHotEncoder()
onehot_encoded = onehot_encoder.fit_transform(nest_list)
print(onehot_encoded)

### Bag of words
    
    Similar to one-hot encoding, Bag of word maps to unique integer is between 1 & |V|. Each document in the corpus converted into a vector of |V| dimention. where in the ith component of the vector simply the number od times the word w occurs in the document. Each word in the V by thier occurrences count in the document.
    
    EX: Vocab =  [i =1, read=2, newspaper=3, yesterday=4, today=5, john=6, TV=7, watch=8]
        i read newspaper today. = [1,1,1,0,1,0,0,0]
        i read newspaper today, i watch tv today = [2,1,1,0,2,0,1,1]


- With this method, documents having same words will have thier vector epresentation closer to each other in euclidean space.
- Fixed length of encoding for any sentence of length

- Size of the vector increase == Size of the vocabulary, Restrict by limiting vocabulary
- Doesn''t capture similarity between different words.
- Doesn't handle out of vocabulary words
- Word order information is lost in this representation.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [47]:
print(pre_process_list)

['i read newspaper yesterday', 'i watched tv today', 'john read newspaper and watched tv today']


In [48]:
# Initalize count vectorrizer
count_vect = CountVectorizer()

In [57]:
# Build BOW for the word list
bow = count_vect.fit_transform(pre_process_list)

print(count_vect.vocabulary_)

{'read': 3, 'newspaper': 2, 'yesterday': 7, 'watched': 6, 'tv': 5, 'today': 4, 'john': 1, 'and': 0}


In [58]:
print("i read newspaper yesterday': ", bow[0].toarray())
print("i watched tv today: ",bow[1].toarray())

i read newspaper yesterday':  [[0 0 1 1 0 0 0 1]]
i watched tv today:  [[0 0 0 0 1 1 1 0]]


In [59]:
new_text = count_vect.transform(["i read newspaper today i watch tv today"])
print("i read newspaper today i watch tv today': ", new_text.toarray())

i read newspaper today i watch tv today':  [[0 0 1 1 2 1 0 0]]


In [61]:
#BoW with binary vectors
count_vect = CountVectorizer(binary=True)
bow_rep_bin = count_vect.fit_transform(pre_process_list)
text_2 = count_vect.transform(["i read newspaper today i watch tv today"])
print("i read newspaper today i watch tv today':", text_2.toarray())

i read newspaper today i watch tv today': [[0 0 1 1 1 1 0 0]]


### Bag of N-Grams
    One-hot encoding and bag of words treat words as independent units. and there is no word ordering. The bag od N-grams tries to solve this by breaking text into chunks of n touching words. This will help to capture context. Each chunk is called n-gram. vocabulary is nothing but collection of all unique n-gram.
    
    ex: bigram model - i read newspaper today,i watch tv today - i read, read newspaper, newspaper today, today i, i watch, watch tv, tv today.
    
    - It capture some context and word-order information in the orm of n-grams
    - Because of above it can able to capture some semantic similarity. 
    - As n increases, dimensionality only increases rapidly
    - It doen't address OOV problem (handling out of vocabulary)



In [5]:
count_vect = CountVectorizer(ngram_range=(1,3))

In [6]:
bow = count_vect.fit_transform(pre_process_list)

print(count_vect.vocabulary_)

{'read': 10, 'newspaper': 6, 'yesterday': 20, 'read newspaper': 11, 'newspaper yesterday': 9, 'read newspaper yesterday': 13, 'watched': 17, 'tv': 15, 'today': 14, 'watched tv': 18, 'tv today': 16, 'watched tv today': 19, 'john': 3, 'and': 0, 'john read': 4, 'newspaper and': 7, 'and watched': 1, 'john read newspaper': 5, 'read newspaper and': 12, 'newspaper and watched': 8, 'and watched tv': 2}


In [7]:
print("i read newspaper yesterday': ", bow[0].toarray())
print("i watched tv today: ",bow[1].toarray())

i read newspaper yesterday':  [[0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1]]
i watched tv today:  [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0]]


In [8]:
new_text = count_vect.transform(["i read newspaper today i watch tv today"])
print("i read newspaper today i watch tv today': ", new_text.toarray())

i read newspaper today i watch tv today':  [[0 0 0 0 0 0 1 0 0 0 1 1 0 0 2 1 1 0 0 0 0]]


### TF-IDF
     In all other approaches text are treated as important. there is no impression of some words in the document being more important than others. TF-IDF (Term Frequenct-inverse document frequency ) solv this problem. It try to quantify the importance of a given word relative to other words. it commonly used in information-retriveal system. 
     
     Idea behind TF-IDF is if the word "W" apperas in many times in doument A and not occur much in document B. then word "W" is much important to document A.
     
     TF - measures how often a term or word occurs in given document. This may give biased results when comes to longer documents. to resolve that the number of occurance divided by the length of the doument.
     
     IDF - Measure the importance of the term across a corpus. when computing TF all terms are given equal importance.but stop words are very common and occur many times in the document and those words are not important. So IDF weighs down the terms that are very common across coprus and weighs up the rare terms.
     
     IDF = loge(total number of documents in the corpus) (Number of documents with term t in them)
     
     TF-IDF score  = TF * IDF
     
     Even though it perform better compare to other methods still suffers from the curs of high dimensionality.
     


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfidf = TfidfVectorizer()

In [11]:
bow_tfidf = tfidf.fit_transform(pre_process_list)

In [12]:
#IDF for all words in the vocabulary
print("IDF for all words in the vocabulary",tfidf.idf_)

#All words in the vocabulary.
print("All words in the vocabulary",tfidf.get_feature_names())


IDF for all words in the vocabulary [1.69314718 1.69314718 1.28768207 1.28768207 1.28768207 1.28768207
 1.28768207 1.69314718]
All words in the vocabulary ['and', 'john', 'newspaper', 'read', 'today', 'tv', 'watched', 'yesterday']


In [14]:
print("TFIDF representation for all documents in our corpus\n",bow_tfidf.toarray()) 

TFIDF representation for all documents in our corpus
 [[0.         0.         0.51785612 0.51785612 0.         0.
  0.         0.68091856]
 [0.         0.         0.         0.         0.57735027 0.57735027
  0.57735027 0.        ]
 [0.45212331 0.45212331 0.34385143 0.34385143 0.34385143 0.34385143
  0.34385143 0.        ]]


In [15]:
new_text = tfidf.transform(["i read newspaper today i watch tv today"])
print("i read newspaper today i watch tv today': ", new_text.toarray())

i read newspaper today i watch tv today':  [[0.         0.         0.37796447 0.37796447 0.75592895 0.37796447
  0.         0.        ]]


## Distributed Representations
    There are some key drawbacks in the vectorization method. To over come that method to learn low-dimentional representation were devised. 
    they use nural netwrok architecture to create dense, low dimnetional representation of words and texts. 
    
### Distributional similarity
    This is called meaning understood by context. ex: "MJ Rocks" - Rocks literally meaning stones but in this context it means good.
    
### Distributional hypothesis
    Word that occur in similar context have similar meanings. cat, mouse. is having similar context that's animal and their characterstics. according to distribuationl hypothesis there should be strong similarities between the meaning of these two words.
    
### Distributional representation
    
    
### Distributed representation


### Embedding
    Embedding is a mapping between vector space coming from distibutional representation to vector space coming from distributed representation.

### Vector semantics
    Set of NLP methods that aim to learn the word representations based on distributional properties words in a large corpus.
    


## Word Embessings
    Lets just say we give word "Cat" distributionally similar words could be other animals either domostic or wild animals.  Neural network word2vec model based on distributional similarity can capture word analogy relationship "king-man+woman = queen". when we learn semantically rich relationships word2vec ensures that the learned word representation are low dimentional.Thease representaion are called embeddings.
    
    Conceptually, Word2vec takes a large corpus of text as input and “learns” to represent the words in a common vector space based on the contexts in which they appear in the corpus. Given a word w and the words appearing in its context C, how do we find the vector that best represents the meaning of the word? For every word w in corpus, we start with a vector v initialized with random values. The Word2vec model refines the values in v by predicting v , given the vectors for words in the context C. It does this using a two-layer neural network.
    

## Pre-Trained Word embeddings
    Train our own embedding is pretty expensive in both time and computing. Some one already trained embedding on large cporpus such as  wikipedia, news article or entier web. these contain key value pair where key represent the words and value represents corresponding word vectors. 